# Final Project 
### Optimizing Inventory by Anticipating Error in Demand Forecast
Felix, Jack, Silvano


### Dataset - Synthetic Store Operations
[Kaggle Dataset](https://www.kaggle.com/datasets/anirudhchauhan/retail-store-inventory-forecasting-dataset?resource=download)


1. Describe the dataset you chose. Why did you choose it? What features does it include? What year is it from? How was it collected? What should we know about this dataset as we read your writeup? (8pts)

    **We are interested in trying to optimize supply chain operations through creating ideal prediction points to order products that keep stores in overstocked rather than understocked, potentially hurting sales.**

    **We chose this dataset because we were interested in the operations side of Data Science and this dataset is robust.**

    **This is a time-series dataset that looks at the following by the day:**
    - Store ID
    - Region
    - Product ID
    - Product Type
    - Inventory Level
    - Units Sold
    - Demand Forecast 
    - Units Ordered
    - Price of Goods
    - Discount Amount
    - Weather Conditions
    - Holiday/Promotions (boolean)
    - Competitor Pricing
    - Seasonality
   
    **Per the dataset's Kaggle home page the dataset was posted ~4 months ago in 2025. It was sythetically generated so we are not working with real-world data. This gives us freedom to use this data without much restriction.**

    **There are also some inconsisitencies we encountered [elaborate], which caused us to trim back the data more than we had initially wanted to.**
    

## Defining a Research Question

2. Define a research question. What are you trying to predict? Describe what you’re trying to accomplish (it will differ between Supervised and Unsupervised learning). (4pts)
    
    Research Question
-   **How can we optimize inventory levels for Store S001 in the East region by using the best supervised model out of ARIMA, Random Forest Ensemble method and XG Boost Regression?**

    **Initially, we wanted to simply predict inventory level and figure out the best model to forecast future levels so we could decide prediction points to strategically order items. We quickly realized there is already a "Demand Forecast" that this data comes with and pivoted to trying to predict the error of the demand vs sales. This would let us know if the dataset's built in predictions could be optimized.**

    **This way, if we have predictions for the future and what kind of error they may be, we can adjust ordering depending on over or under predictions.**


3. Why is this algorithm a good way of answering your research question? (4pts)
    >   
        Hayes, A. (2010, April 11). Autoregressive integrated moving average (ARIMA) prediction model. Investopedia. https://www.investopedia.com/terms/a/autoregressive-integrated-moving-average-arima.asp

    - **The ARIMA model is a powerhouse when dealing with time-series data. It has the innate capability to analyze moving averages and incoporates this into its package. When we trained the XGBoost model we had to manually engineer features to give it mathematical seaonsality.** 

4. Using the data you chose and the algorithm you chose, read in your data and run your model. (10pts)
# ENTER PROCESS HERE - for felix to enter

5. Conduct a hyperparameter sensitivity analysis by systematically varying key model parameters and measuring their impact on model performance. You only need to do this for one hyperparameter if your selected algorithm has multiple. (6pts)

# SHOW HYPER PARAMETER TUNING HERE - for felix to enter


6. Report the evaluation of your model. Either through RMSE, F1 score, or another metric. How did accuracy/evaluation change with hyperparameter selection? (6pts)



7. Create a visualization demonstrating your findings. Make sure to include a title and axis labels. Describe what’s being shown in your visualization. (8pts)


8. What challenges did you run into? Do you think it was because of the data, the model, the research question? How would you overcome these challenges? (6pts)

    A big challenge at the beginning was trying to figure out what data to include in our model. The data was synthetically generated resulting in some columns causing more confusion than necessary. We first wanted to predict inventory levels for a particular product, then extend our findings to all products. We found that the products overlapped in terms of product ID and where they were they were sold. We ended up dropping a lot of the unecessary and redundant columns. Things like 



9. Explain how your machine learning solution answers the research question you defined. Describe the risks and advantages of applying your solution to unseen data. What are the consequences of your solution being wrong? What are advantages when it is right?  What are some negative and positive societal impacts from your model? (10pts)



10. Next Research Question. Why is it important? (3pts)



11. Model Card